# Usable Data Map (UDM2) Cloud Detection within an AOI

This guide is a follow up to the [UDM2 Cloud Detection](udm2_clouds.ipynb) notebook. Please refer to that notebook for further details on specifications and usage of the UDM2 asset. In this notebook, we apply cloud detection to a specific area of interest (AOI).

The `ortho_udm2` and `udm2` assets are available for all `PSScene` 4-Band and `PSOrthoTile` items created after 2018-08-01, respectively. Therefore, our search should be limited to these items and this date range. In this notebook, we focus on `PSOrthoTile` imagery taken within the month of April, 2019.

## Import dependencies

In [ ]:
import datetime
import json
import os
import time

from datetime import datetime
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter
import rasterio
from rasterio.plot import show
import requests

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

## Finding clear imagery

One of the benefits of accurate and automated cloud detection is that it allows users to filter out images that don't meet a certain quality threshold. Planet's Data API allows users to [search](https://developers.planet.com/docs/apis/data/searches-filtering/) based on the value of the imagery metadata.
    
Planet's cloud detection algorithm classifies every pixel into one of six different categories, each of which has a corresponding metadata field that reflects the percentage of data that falls into the category.

| Class | Metadata field |
| --- | --- |
| clear | `clear_percent` |
| snow | `snow_ice_percent` |
| shadow | `shadow_percent` |
| light haze | `light_haze_percent` |
| heavy haze| `heavy_haze_percent` |
| cloud | `cloud_percent` |

The [UDM2 Cloud Detection](udm2_clouds.ipynb) notebook provides examples for how to perform searches using the Planet Python Client command-line interface.

The following example will show how use Planet's Python Client to perform a search for PSOrthoTiles that are 70-95% clear (not totally clear so the udm has some interesting content), taken within the month of April, 2019, and within an AOI.

In [ ]:
# define the aoi for imagery
aoi = {"geometry": {
    "type":"Polygon",
    "coordinates":
        [[
            [25.42429478260258,1.0255377823058893],
            [25.592960813580472,1.0255377823058893],
            [25.592960813580472,1.1196578801254304],
            [25.42429478260258,1.1196578801254304],
            [25.42429478260258,1.0255377823058893]
        ]]}}


item_types = ['PSOrthoTile']

In [ ]:
# filter by our gemoetry defined above
geom_filter = data_filter.geometry_filter(aoi)

# clear_percent between 70 and 95, inclusive
clear_percent_filter = data_filter.range_filter('clear_percent', None, None, 70, 95)

# imagery acquired between April 1st and May 1st 2019
date_range_filter = data_filter.date_range_filter("acquired", datetime(month=4, day=1, year=2019), datetime(month=5, day=1, year=2019))

# combine all these filters together
combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter])

In [ ]:
async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='udm2_clouds_aoi',search_filter=combined_filter, item_types=item_types)

In [ ]:
print (request)

In [ ]:
# Search the Data API
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.run_search(search_id=request['id'])
    item_list = [i async for i in items]

In [ ]:
print(len(item_list))

In [ ]:
# Let's look at the first item in our results list
item = item_list[0]
item_id = item_list[0]['id']
item_id

## The `udm2` asset

In addition to metadata for filtering, the `udm2` asset provides a pixel-by-pixel map that identifies the classification of each pixel. See the [UDM2 Cloud Detection](udm2_clouds.ipynb) notebook for an example map.

The `udm2` structure is to use a separate band for each classification type. Band 2, for example, indicates that a pixel is snowy when its value is 1, band 3 indicates shadow and so on. "

The following Python will download the data above and then display pixels that fall into a certain classifications.

First, we need to activate the asset that we've identified:

In [ ]:
# Assign a variable to the udm2 asset from the item's assets
udm2_asset = assets["udm2"]
analytic_asset = assets["analytic"]

# Setup the activation url for a particular asset (in this case an analytic asset)
activation_url_udm2 = udm2_asset["_links"]["activate"]
activation_url_analytic = analytic_asset["_links"]["activate"]

# Send a request to the activation url to activate the item
res_udm2 = session.get(activation_url_udm2)
res_analytic = session.get(activation_url_analytic)

# Print the response from the activation request
print(res_udm2.status_code)
print(res_analytic.status_code)

A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

Below, we are polling the API until the item is done activation. This may take awhile.

In [ ]:
# Get the assets link for the item
assets_url = item["_links"]["assets"]

# Poll the API until the item is done activating
assets_activated = False

while assets_activated == False:
    # Send a request to the item's assets url
    res = session.get(assets_url)

    # Assign a variable to the item's assets url response
    assets = res.json()

    udm2_asset_status = udm2_asset["status"]
    analytic_asset_status = analytic_asset["status"]
    
    print(udm2_asset_status, analytic_asset_status)
    
    time.sleep(15) 
    
    # If both assets are already active, we are done
    if (udm2_asset_status == 'active') & (analytic_asset_status == 'active'):
        assets_activated = True
        print("Assets are active and ready to download")

# Print the udm2 asset data    
print(udm2_asset)
print(analytic_asset)

active inactive
active inactive
active inactive
active inactive


In [ ]:
# start downloads
data_dir = 'data'
r1 = client.download(assets["analytic"], callback=api.write_to_file(data_dir))
r2 = client.download(assets["udm2"], callback=api.write_to_file(data_dir))

In [ ]:
# wait until downloads complete
r1.wait()
r2.wait()
img_file = os.path.join(data_dir, r1.get_body().name)
udm_file = os.path.join(data_dir, r2.get_body().name)
print("image: {}".format(img_file))
print("udm2:  {}".format(udm_file))

### Visualize Image and UDM2

In [ ]:
with rasterio.open(udm_file) as src:
    shadow_mask = src.read(3).astype(bool)
    cloud_mask = src.read(6).astype(bool)
    
show(shadow_mask, title="shadow", cmap="binary")
show(cloud_mask, title="cloud", cmap="binary")

In [ ]:
mask = shadow_mask + cloud_mask
show(mask, title="mask", cmap="binary")

In [ ]:
with rasterio.open(img_file) as src:
    profile = src.profile
    img_data = src.read([3, 2, 1], masked=True) / 10000.0 # apply RGB ordering and scale down

In [ ]:
show(img_data, title=item_id)

In [ ]:
img_data.mask = mask
img_data = img_data.filled(fill_value=0)

In [ ]:
show(img_data, title="masked image")

The image stored in `img_data` now has cloudy / cloud-shadowy pixels masked out and can be saved or used for analysis.